# Summary:
- Imbalanced sampling (SMOTE) is applied.
- Below networks are tested with below results:
    - SVM 
                             precision    recall  f1-score   support
                0.0              0.79      0.82      0.80      2129
                1.0              0.82      0.78      0.80      2166
                avg / total      0.80      0.80      0.80      4295
                
                Confusion Matrix
                    1746  383
                    466   1700

    - DNN + SVM
                             precision    recall  f1-score   support
                0.0              0.91      0.96      0.93      2129
                1.0              0.96      0.91      0.93      2166
                avg / total      0.94      0.93      0.93      4295
                
                Confusion Matrix
                    2046   83
                    202    1964
               
    - Deeper DNN with dropout + SVM
                             precision    recall  f1-score   support
                0.0              0.93      0.92      0.93      2129
                1.0              0.93      0.94      0.93      2166
                avg / total      0.93      0.93      0.93      4295
                
                Confusion Matrix
                    1967  162
                    138   2028
    
    - Inception Batch Normalization + SVM
                             precision    recall  f1-score   support
                0.0              0.93      0.94      0.94      2129
                1.0              0.94      0.93      0.94      2166
                avg / total      0.94      0.94      0.94      4295
                
                Confusion Matrix
                    2004  125
                    141   2025

In [1]:
import glob
import multiprocessing
import time
import csv
import pickle
import os
import pandas as pd
import shutil
import numpy as np
import warnings
import tqdm
import logging
import mxnet as mx
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import basename
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE, ADASYN

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/mxnet/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Populating the interactive namespace from numpy and matplotlib


In [2]:
# 2d to 4d conversion
def to4d(img):
    return img.reshape(img.shape[0], 1, 32, 32).astype(np.float32)

In [3]:
def modelMetric(mod, info, nd=True):
    if nd == True: 
        prob = mod.predict(val_iters).asnumpy()
    else:
        prob = mod.predict(val_iters)
    logging.info(info)
    # Check the accuracy from prediction
    val_iters.reset()

    y_batch = []
    for dbatch in val_iters:
        label = dbatch.label[0].asnumpy()
        pad = val_iters.getpad()
        real_size = label.shape[0] - pad
        y_batch.append(label[0:real_size])
    y = np.concatenate(y_batch)

    # get prediction label 
    py = np.argmax(prob, axis=1)
    acc1 = sum(py == y) / len(y)
    logging.info('Test accuracy = %f', acc1)
    cfn_matrix=confusion_matrix(y,py)
    print(classification_report (y,py))
    print(cfn_matrix)

# 1. Data Preparation

In [4]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [5]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [6]:
# Use SMOTE oversampling to generate more training data
X_resampled, y_resampled = SMOTE().fit_sample(vectors, data_Y)

In [7]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(X_resampled, y_resampled, test_size=0.25)

In [8]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [9]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((12883, 1024), (12883,), (4295, 1024), (4295,), 10216)

In [10]:
# creating 4d training/testing dataset
batch_size = 64
train_iters = mx.io.NDArrayIter(to4d(X_train_array), Y_train_array, batch_size, shuffle=True,label_name='svm_label')
val_iters = mx.io.NDArrayIter(to4d(X_test_array), Y_test_array, batch_size,label_name='svm_label')

# 2. Train models
## 2.1 SVM

In [11]:
# Single SVM
data = mx.symbol.Variable('data')
svm  = mx.symbol.FullyConnected(data = data, name='fc', num_hidden=2)

# Here we add the ultimate layer based on L2-SVM objective
ssvm = mx.symbol.SVMOutput(data=svm, name='svm')
# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart.format = 'png'
network_chart.render('SVM')

<img src='SVM.png'>

In [12]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

mod = mx.model.FeedForward(
    ctx = mx.gpu(0),     # use GPU 0 for training, others are same as before
    symbol = ssvm,       # network structure
    num_epoch = 100,     # number of data passes for training 
    learning_rate = 0.01 # learning rate of SGD 
)
mod.fit(
    X=train_iters,       # training data
    eval_data=val_iters, # validation data
    batch_end_callback = mx.callback.Speedometer(batch_size, 64) # output progress for each 200 data batches
)


INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [64]	Speed: 108456.72 samples/sec	accuracy=0.688477
INFO:root:Epoch[0] Batch [128]	Speed: 98074.85 samples/sec	accuracy=0.747559
INFO:root:Epoch[0] Batch [192]	Speed: 102633.17 samples/sec	accuracy=0.766357
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.133
INFO:root:Epoch[0] Validation-accuracy=0.768382
INFO:root:Epoch[1] Batch [64]	Speed: 103805.23 samples/sec	accuracy=0.782715
INFO:root:Epoch[1] Batch [128]	Speed: 94851.42 samples/sec	accuracy=0.791504
INFO:root:Epoch[1] Batch [192]	Speed: 106251.90 samples/sec	accuracy=0.786377
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.134
INFO:root:Epoch[1] Validation-accuracy=0.780561
INFO:root:Epoch[2] Batch [64]	Speed: 103088.29 samples/sec	accuracy=0.796631
INFO:root:Epoch[2] Batch [128]	Speed: 121057.46 samples/sec	accuracy=0.800293
INFO:root:Epoch[2] Batch [192]	Speed: 130627.51 samples/sec	accuracy=0.794434
INFO:root:Ep

INFO:root:Epoch[22] Resetting Data Iterator
INFO:root:Epoch[22] Time cost=0.095
INFO:root:Epoch[22] Validation-accuracy=0.795726
INFO:root:Epoch[23] Batch [64]	Speed: 156669.15 samples/sec	accuracy=0.815430
INFO:root:Epoch[23] Batch [128]	Speed: 150957.50 samples/sec	accuracy=0.815430
INFO:root:Epoch[23] Batch [192]	Speed: 156789.26 samples/sec	accuracy=0.813721
INFO:root:Epoch[23] Resetting Data Iterator
INFO:root:Epoch[23] Time cost=0.088
INFO:root:Epoch[23] Validation-accuracy=0.795496
INFO:root:Epoch[24] Batch [64]	Speed: 154448.00 samples/sec	accuracy=0.815674
INFO:root:Epoch[24] Batch [128]	Speed: 153744.48 samples/sec	accuracy=0.815674
INFO:root:Epoch[24] Batch [192]	Speed: 153101.89 samples/sec	accuracy=0.812988
INFO:root:Epoch[24] Resetting Data Iterator
INFO:root:Epoch[24] Time cost=0.088
INFO:root:Epoch[24] Validation-accuracy=0.795037
INFO:root:Epoch[25] Batch [64]	Speed: 138832.84 samples/sec	accuracy=0.816162
INFO:root:Epoch[25] Batch [128]	Speed: 164400.66 samples/sec	ac

INFO:root:Epoch[45] Batch [128]	Speed: 153799.53 samples/sec	accuracy=0.821045
INFO:root:Epoch[45] Batch [192]	Speed: 156187.73 samples/sec	accuracy=0.817383
INFO:root:Epoch[45] Resetting Data Iterator
INFO:root:Epoch[45] Time cost=0.088
INFO:root:Epoch[45] Validation-accuracy=0.798483
INFO:root:Epoch[46] Batch [64]	Speed: 109875.92 samples/sec	accuracy=0.820068
INFO:root:Epoch[46] Batch [128]	Speed: 97861.44 samples/sec	accuracy=0.821533
INFO:root:Epoch[46] Batch [192]	Speed: 91460.61 samples/sec	accuracy=0.817871
INFO:root:Epoch[46] Resetting Data Iterator
INFO:root:Epoch[46] Time cost=0.135
INFO:root:Epoch[46] Validation-accuracy=0.799173
INFO:root:Epoch[47] Batch [64]	Speed: 103939.63 samples/sec	accuracy=0.820557
INFO:root:Epoch[47] Batch [128]	Speed: 105303.65 samples/sec	accuracy=0.821533
INFO:root:Epoch[47] Batch [192]	Speed: 124365.64 samples/sec	accuracy=0.817871
INFO:root:Epoch[47] Resetting Data Iterator
INFO:root:Epoch[47] Time cost=0.121
INFO:root:Epoch[47] Validation-acc

INFO:root:Epoch[67] Time cost=0.083
INFO:root:Epoch[67] Validation-accuracy=0.800551
INFO:root:Epoch[68] Batch [64]	Speed: 159136.22 samples/sec	accuracy=0.820068
INFO:root:Epoch[68] Batch [128]	Speed: 158309.17 samples/sec	accuracy=0.822510
INFO:root:Epoch[68] Batch [192]	Speed: 137868.00 samples/sec	accuracy=0.818604
INFO:root:Epoch[68] Resetting Data Iterator
INFO:root:Epoch[68] Time cost=0.090
INFO:root:Epoch[68] Validation-accuracy=0.800551
INFO:root:Epoch[69] Batch [64]	Speed: 158166.34 samples/sec	accuracy=0.820312
INFO:root:Epoch[69] Batch [128]	Speed: 158750.95 samples/sec	accuracy=0.822510
INFO:root:Epoch[69] Batch [192]	Speed: 156843.65 samples/sec	accuracy=0.818604
INFO:root:Epoch[69] Resetting Data Iterator
INFO:root:Epoch[69] Time cost=0.088
INFO:root:Epoch[69] Validation-accuracy=0.800551
INFO:root:Epoch[70] Batch [64]	Speed: 147464.16 samples/sec	accuracy=0.820312
INFO:root:Epoch[70] Batch [128]	Speed: 142576.26 samples/sec	accuracy=0.822510
INFO:root:Epoch[70] Batch [1

INFO:root:Epoch[90] Batch [128]	Speed: 168289.85 samples/sec	accuracy=0.823730
INFO:root:Epoch[90] Batch [192]	Speed: 166539.38 samples/sec	accuracy=0.817627
INFO:root:Epoch[90] Resetting Data Iterator
INFO:root:Epoch[90] Time cost=0.088
INFO:root:Epoch[90] Validation-accuracy=0.801700
INFO:root:Epoch[91] Batch [64]	Speed: 124101.52 samples/sec	accuracy=0.821777
INFO:root:Epoch[91] Batch [128]	Speed: 108836.68 samples/sec	accuracy=0.823730
INFO:root:Epoch[91] Batch [192]	Speed: 100779.43 samples/sec	accuracy=0.817627
INFO:root:Epoch[91] Resetting Data Iterator
INFO:root:Epoch[91] Time cost=0.122
INFO:root:Epoch[91] Validation-accuracy=0.801700
INFO:root:Epoch[92] Batch [64]	Speed: 102736.90 samples/sec	accuracy=0.822021
INFO:root:Epoch[92] Batch [128]	Speed: 112842.83 samples/sec	accuracy=0.823975
INFO:root:Epoch[92] Batch [192]	Speed: 94900.67 samples/sec	accuracy=0.817627
INFO:root:Epoch[92] Resetting Data Iterator
INFO:root:Epoch[92] Time cost=0.131
INFO:root:Epoch[92] Validation-ac

In [13]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'SVM'
epoch = 100
mod.save(prefix, epoch)

INFO:root:Saved checkpoint to "SVM-0100.params"


In [14]:
modelMetric(mod, 'Finish predict of one Layer SVM Network...', False)

INFO:root:Finish predict of one Layer SVM Network...
INFO:root:Test accuracy = 0.802328


             precision    recall  f1-score   support

        0.0       0.79      0.82      0.80      2129
        1.0       0.82      0.78      0.80      2166

avg / total       0.80      0.80      0.80      4295

[[1746  383]
 [ 466 1700]]


## 2.2 DNN + SVM

In [15]:
# DNN + SVM 
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=512)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")

fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=2)

dnnsvm = mx.symbol.SVMOutput(data=fc3, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart.format = 'png'
network_chart.render('DNN + SVM')

'DNN + SVM.pdf'

<img src='DNN + SVM.png'>

In [16]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = dnnsvm,        
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size), 
    num_epoch = 100,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.001,         # Weight decay for regularization
    },
)
print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 49025.81 samples/sec	accuracy=0.494231
INFO:root:Epoch[0] Batch [128]	Speed: 49267.21 samples/sec	accuracy=0.581299
INFO:root:Epoch[0] Batch [192]	Speed: 53169.68 samples/sec	accuracy=0.588867
INFO:root:Epoch[0] Train-accuracy=0.708333
INFO:root:Epoch[0] Time cost=0.275
INFO:root:Epoch[0] Validation-accuracy=0.753447
INFO:root:Epoch[1] Batch [64]	Speed: 56325.78 samples/sec	accuracy=0.702163
INFO:root:Epoch[1] Batch [128]	Speed: 53798.56 samples/sec	accuracy=0.686035
INFO:root:Epoch[1] Batch [192]	Speed: 61405.80 samples/sec	accuracy=0.721680
INFO:root:Epoch[1] Train-accuracy=0.739583
INFO:root:Epoch[1] Time cost=0.231
INFO:root:Epoch[1] Validation-accuracy=0.760340
INFO:root:Epoch[2] Batch [64]	Speed: 56527.79 samples/sec	accuracy=0.768750
INFO:root:Epoch[2] Batch [128]	Speed: 56201.03 samples/sec	accuracy=0.798096
INFO:root:Epoch[2] Batch [192]	Speed: 62203.31 samples/sec	accuracy=0.816650
INFO:root:Epoch[2] Train-accuracy=0.800347
INFO:root:Epoch

INFO:root:Epoch[22] Validation-accuracy=0.923943
INFO:root:Epoch[23] Batch [64]	Speed: 74162.08 samples/sec	accuracy=0.997115
INFO:root:Epoch[23] Batch [128]	Speed: 77610.89 samples/sec	accuracy=0.995850
INFO:root:Epoch[23] Batch [192]	Speed: 83258.79 samples/sec	accuracy=0.997803
INFO:root:Epoch[23] Train-accuracy=0.998264
INFO:root:Epoch[23] Time cost=0.170
INFO:root:Epoch[23] Validation-accuracy=0.930607
INFO:root:Epoch[24] Batch [64]	Speed: 77580.40 samples/sec	accuracy=0.997115
INFO:root:Epoch[24] Batch [128]	Speed: 79590.23 samples/sec	accuracy=0.995850
INFO:root:Epoch[24] Batch [192]	Speed: 77418.88 samples/sec	accuracy=0.998291
INFO:root:Epoch[24] Train-accuracy=0.998264
INFO:root:Epoch[24] Time cost=0.168
INFO:root:Epoch[24] Validation-accuracy=0.932675
INFO:root:Epoch[25] Batch [64]	Speed: 76988.33 samples/sec	accuracy=0.997837
INFO:root:Epoch[25] Batch [128]	Speed: 78361.37 samples/sec	accuracy=0.995850
INFO:root:Epoch[25] Batch [192]	Speed: 84308.45 samples/sec	accuracy=0.9

INFO:root:Epoch[45] Train-accuracy=1.000000
INFO:root:Epoch[45] Time cost=0.225
INFO:root:Epoch[45] Validation-accuracy=0.927619
INFO:root:Epoch[46] Batch [64]	Speed: 73133.21 samples/sec	accuracy=0.999519
INFO:root:Epoch[46] Batch [128]	Speed: 78901.57 samples/sec	accuracy=0.999512
INFO:root:Epoch[46] Batch [192]	Speed: 78970.11 samples/sec	accuracy=0.999756
INFO:root:Epoch[46] Train-accuracy=1.000000
INFO:root:Epoch[46] Time cost=0.172
INFO:root:Epoch[46] Validation-accuracy=0.929917
INFO:root:Epoch[47] Batch [64]	Speed: 35440.39 samples/sec	accuracy=0.999519
INFO:root:Epoch[47] Batch [128]	Speed: 52853.66 samples/sec	accuracy=0.999512
INFO:root:Epoch[47] Batch [192]	Speed: 80468.15 samples/sec	accuracy=0.999756
INFO:root:Epoch[47] Train-accuracy=1.000000
INFO:root:Epoch[47] Time cost=0.257
INFO:root:Epoch[47] Validation-accuracy=0.929917
INFO:root:Epoch[48] Batch [64]	Speed: 81716.29 samples/sec	accuracy=0.999519
INFO:root:Epoch[48] Batch [128]	Speed: 80110.18 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [192]	Speed: 80177.48 samples/sec	accuracy=1.000000
INFO:root:Epoch[68] Train-accuracy=1.000000
INFO:root:Epoch[68] Time cost=0.177
INFO:root:Epoch[68] Validation-accuracy=0.930836
INFO:root:Epoch[69] Batch [64]	Speed: 75263.16 samples/sec	accuracy=0.999760
INFO:root:Epoch[69] Batch [128]	Speed: 76693.11 samples/sec	accuracy=1.000000
INFO:root:Epoch[69] Batch [192]	Speed: 79132.34 samples/sec	accuracy=1.000000
INFO:root:Epoch[69] Train-accuracy=1.000000
INFO:root:Epoch[69] Time cost=0.174
INFO:root:Epoch[69] Validation-accuracy=0.929228
INFO:root:Epoch[70] Batch [64]	Speed: 85968.12 samples/sec	accuracy=1.000000
INFO:root:Epoch[70] Batch [128]	Speed: 75177.53 samples/sec	accuracy=1.000000
INFO:root:Epoch[70] Batch [192]	Speed: 83130.68 samples/sec	accuracy=1.000000
INFO:root:Epoch[70] Train-accuracy=1.000000
INFO:root:Epoch[70] Time cost=0.163
INFO:root:Epoch[70] Validation-accuracy=0.929228
INFO:root:Epoch[71] Batch [64]	Speed: 75247.00 samples/sec	accuracy=1

INFO:root:Epoch[91] Batch [128]	Speed: 72970.44 samples/sec	accuracy=1.000000
INFO:root:Epoch[91] Batch [192]	Speed: 76735.24 samples/sec	accuracy=1.000000
INFO:root:Epoch[91] Train-accuracy=1.000000
INFO:root:Epoch[91] Time cost=0.176
INFO:root:Epoch[91] Validation-accuracy=0.930836
INFO:root:Epoch[92] Batch [64]	Speed: 78661.32 samples/sec	accuracy=1.000000
INFO:root:Epoch[92] Batch [128]	Speed: 79244.77 samples/sec	accuracy=1.000000
INFO:root:Epoch[92] Batch [192]	Speed: 77765.11 samples/sec	accuracy=1.000000
INFO:root:Epoch[92] Train-accuracy=1.000000
INFO:root:Epoch[92] Time cost=0.168
INFO:root:Epoch[92] Validation-accuracy=0.930607
INFO:root:Epoch[93] Batch [64]	Speed: 77164.34 samples/sec	accuracy=1.000000
INFO:root:Epoch[93] Batch [128]	Speed: 82667.06 samples/sec	accuracy=1.000000
INFO:root:Epoch[93] Batch [192]	Speed: 82103.30 samples/sec	accuracy=1.000000
INFO:root:Epoch[93] Train-accuracy=1.000000
INFO:root:Epoch[93] Time cost=0.164
INFO:root:Epoch[93] Validation-accuracy=

Accuracy: 93.33639705882352 %


In [17]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DNN_SVM'
epoch = 100
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DNN_SVM-0100.params"


In [18]:
modelMetric(mod, 'Finish predict of DNN+SVM...')

INFO:root:Finish predict of DNN+SVM...
INFO:root:Test accuracy = 0.933644


             precision    recall  f1-score   support

        0.0       0.91      0.96      0.93      2129
        1.0       0.96      0.91      0.93      2166

avg / total       0.94      0.93      0.93      4295

[[2046   83]
 [ 202 1964]]


## 2.3 Deeper DNN with dropout + SVM

In [19]:
# Deeper DNN with dropout + SVM
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=1024)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 1024)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
drop1 = mx.sym.Dropout(data=act2, p=0.5)

fc3  = mx.symbol.FullyConnected(data = drop1, name = 'fc3', num_hidden = 512)
act3 = mx.symbol.Activation(data = fc3, name='relu3', act_type="relu")
fc4  = mx.symbol.FullyConnected(data = act3, name = 'fc4', num_hidden = 512)
act4 = mx.symbol.Activation(data = fc4, name='relu4', act_type="relu")

drop2 = mx.sym.Dropout(data=act4, p=0.5)

fc5  = mx.symbol.FullyConnected(data = drop2, name='fc5', num_hidden=2)

ddnn = mx.symbol.SVMOutput(data=fc5, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart.format = 'png'
network_chart.render('DDNN + SVM')

'DDNN + SVM.pdf'

<img src='DDNN + SVM.png'>

In [20]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ddnn,         # Use the network we just defined
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, 64),  
    num_epoch = 100,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 34154.67 samples/sec	accuracy=0.495913
INFO:root:Epoch[0] Batch [128]	Speed: 35078.28 samples/sec	accuracy=0.494629
INFO:root:Epoch[0] Batch [192]	Speed: 37234.14 samples/sec	accuracy=0.497314
INFO:root:Epoch[0] Train-accuracy=0.519097
INFO:root:Epoch[0] Time cost=0.406
INFO:root:Epoch[0] Validation-accuracy=0.496324
INFO:root:Epoch[1] Batch [64]	Speed: 35882.90 samples/sec	accuracy=0.494471
INFO:root:Epoch[1] Batch [128]	Speed: 37812.64 samples/sec	accuracy=0.496582
INFO:root:Epoch[1] Batch [192]	Speed: 38229.22 samples/sec	accuracy=0.497559
INFO:root:Epoch[1] Train-accuracy=0.519097
INFO:root:Epoch[1] Time cost=0.347
INFO:root:Epoch[1] Validation-accuracy=0.496324
INFO:root:Epoch[2] Batch [64]	Speed: 37578.73 samples/sec	accuracy=0.494712
INFO:root:Epoch[2] Batch [128]	Speed: 38849.15 samples/sec	accuracy=0.496582
INFO:root:Epoch[2] Batch [192]	Speed: 38159.51 samples/sec	accuracy=0.497314
INFO:root:Epoch[2] Train-accuracy=0.519097
INFO:root:Epoch

INFO:root:Epoch[22] Validation-accuracy=0.799173
INFO:root:Epoch[23] Batch [64]	Speed: 40116.73 samples/sec	accuracy=0.766587
INFO:root:Epoch[23] Batch [128]	Speed: 40426.65 samples/sec	accuracy=0.775635
INFO:root:Epoch[23] Batch [192]	Speed: 41072.16 samples/sec	accuracy=0.768066
INFO:root:Epoch[23] Train-accuracy=0.769097
INFO:root:Epoch[23] Time cost=0.322
INFO:root:Epoch[23] Validation-accuracy=0.783318
INFO:root:Epoch[24] Batch [64]	Speed: 39911.23 samples/sec	accuracy=0.793990
INFO:root:Epoch[24] Batch [128]	Speed: 40451.20 samples/sec	accuracy=0.820312
INFO:root:Epoch[24] Batch [192]	Speed: 41185.49 samples/sec	accuracy=0.836426
INFO:root:Epoch[24] Train-accuracy=0.840278
INFO:root:Epoch[24] Time cost=0.321
INFO:root:Epoch[24] Validation-accuracy=0.837086
INFO:root:Epoch[25] Batch [64]	Speed: 40362.25 samples/sec	accuracy=0.858413
INFO:root:Epoch[25] Batch [128]	Speed: 39272.22 samples/sec	accuracy=0.877197
INFO:root:Epoch[25] Batch [192]	Speed: 41171.38 samples/sec	accuracy=0.8

INFO:root:Epoch[45] Train-accuracy=0.993056
INFO:root:Epoch[45] Time cost=0.346
INFO:root:Epoch[45] Validation-accuracy=0.925322
INFO:root:Epoch[46] Batch [64]	Speed: 36794.58 samples/sec	accuracy=0.991587
INFO:root:Epoch[46] Batch [128]	Speed: 37660.95 samples/sec	accuracy=0.991943
INFO:root:Epoch[46] Batch [192]	Speed: 38191.24 samples/sec	accuracy=0.992920
INFO:root:Epoch[46] Train-accuracy=0.993056
INFO:root:Epoch[46] Time cost=0.347
INFO:root:Epoch[46] Validation-accuracy=0.923483
INFO:root:Epoch[47] Batch [64]	Speed: 36777.81 samples/sec	accuracy=0.991587
INFO:root:Epoch[47] Batch [128]	Speed: 37376.87 samples/sec	accuracy=0.991943
INFO:root:Epoch[47] Batch [192]	Speed: 38488.91 samples/sec	accuracy=0.992920
INFO:root:Epoch[47] Train-accuracy=0.993056
INFO:root:Epoch[47] Time cost=0.348
INFO:root:Epoch[47] Validation-accuracy=0.924862
INFO:root:Epoch[48] Batch [64]	Speed: 36093.98 samples/sec	accuracy=0.991587
INFO:root:Epoch[48] Batch [128]	Speed: 38364.28 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [192]	Speed: 40082.01 samples/sec	accuracy=0.992920
INFO:root:Epoch[68] Train-accuracy=0.993056
INFO:root:Epoch[68] Time cost=0.328
INFO:root:Epoch[68] Validation-accuracy=0.927849
INFO:root:Epoch[69] Batch [64]	Speed: 38980.22 samples/sec	accuracy=0.991827
INFO:root:Epoch[69] Batch [128]	Speed: 39609.96 samples/sec	accuracy=0.992188
INFO:root:Epoch[69] Batch [192]	Speed: 39649.63 samples/sec	accuracy=0.992920
INFO:root:Epoch[69] Train-accuracy=0.993056
INFO:root:Epoch[69] Time cost=0.330
INFO:root:Epoch[69] Validation-accuracy=0.928079
INFO:root:Epoch[70] Batch [64]	Speed: 39569.45 samples/sec	accuracy=0.991827
INFO:root:Epoch[70] Batch [128]	Speed: 39354.54 samples/sec	accuracy=0.992188
INFO:root:Epoch[70] Batch [192]	Speed: 39468.64 samples/sec	accuracy=0.992920
INFO:root:Epoch[70] Train-accuracy=0.993056
INFO:root:Epoch[70] Time cost=0.331
INFO:root:Epoch[70] Validation-accuracy=0.927849
INFO:root:Epoch[71] Batch [64]	Speed: 38852.66 samples/sec	accuracy=0

INFO:root:Epoch[91] Batch [128]	Speed: 40044.63 samples/sec	accuracy=0.992188
INFO:root:Epoch[91] Batch [192]	Speed: 40034.46 samples/sec	accuracy=0.993408
INFO:root:Epoch[91] Train-accuracy=0.993056
INFO:root:Epoch[91] Time cost=0.329
INFO:root:Epoch[91] Validation-accuracy=0.929228
INFO:root:Epoch[92] Batch [64]	Speed: 39593.62 samples/sec	accuracy=0.991827
INFO:root:Epoch[92] Batch [128]	Speed: 40067.89 samples/sec	accuracy=0.992188
INFO:root:Epoch[92] Batch [192]	Speed: 39588.24 samples/sec	accuracy=0.993408
INFO:root:Epoch[92] Train-accuracy=0.993056
INFO:root:Epoch[92] Time cost=0.328
INFO:root:Epoch[92] Validation-accuracy=0.929228
INFO:root:Epoch[93] Batch [64]	Speed: 27253.45 samples/sec	accuracy=0.991827
INFO:root:Epoch[93] Batch [128]	Speed: 29617.30 samples/sec	accuracy=0.992188
INFO:root:Epoch[93] Batch [192]	Speed: 39273.03 samples/sec	accuracy=0.993408
INFO:root:Epoch[93] Train-accuracy=0.993056
INFO:root:Epoch[93] Time cost=0.412
INFO:root:Epoch[93] Validation-accuracy=

Accuracy: 92.99172794117648 %


In [21]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DDNN_SVM'
epoch = 100
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DDNN_SVM-0100.params"


In [22]:
modelMetric(mod, 'Finish predict of DDNN + SVM...')

INFO:root:Finish predict of DDNN + SVM...
INFO:root:Test accuracy = 0.930151


             precision    recall  f1-score   support

        0.0       0.93      0.92      0.93      2129
        1.0       0.93      0.94      0.93      2166

avg / total       0.93      0.93      0.93      4295

[[1967  162]
 [ 138 2028]]


## 2.4 Inception Batach Normalization

In [23]:
# Reference: www.mxnet.io
# Basic Conv + BN + ReLU factory
def ConvFactory(data, num_filter, kernel, stride=(1,1), pad=(0, 0), act_type="relu"):
    conv = mx.symbol.Convolution(data=data, workspace=256,
                                 num_filter=num_filter, kernel=kernel, stride=stride, pad=pad)
    bn = mx.symbol.BatchNorm(data=conv)
    act = mx.symbol.Activation(data = bn, act_type=act_type)
    return act

# A Simple Downsampling Factory
def DownsampleFactory(data, ch_3x3):
    conv = ConvFactory(data=data, kernel=(3, 3), stride=(2, 2), num_filter=ch_3x3, pad=(1, 1))
    pool = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(2, 2),pad=(1,1), pool_type='max')
    concat = mx.symbol.Concat(*[conv, pool])
    return concat

# A Simple module
def SimpleFactory(data, ch_1x1, ch_3x3):
    conv1x1 = ConvFactory(data=data, kernel=(1, 1), pad=(0, 0), num_filter=ch_1x1)
    conv3x3 = ConvFactory(data=data, kernel=(3, 3), pad=(1, 1), num_filter=ch_3x3)
    concat = mx.symbol.Concat(*[conv1x1, conv3x3])
    return concat

def InceptionFactory(data, num_1x1, num_3x3red, num_3x3, num_d5x5red, num_d5x5, pool, proj, name):
    c1x1 = ConvFactoryB(data=data, num_filter=num_1x1, kernel=(1, 1), name=('%s_1x1' % name))
    c3x3r = ConvFactoryB(data=data, num_filter=num_3x3red, kernel=(1, 1), name=('%s_3x3' % name), suffix='_reduce')
    c3x3 = ConvFactoryB(data=c3x3r, num_filter=num_3x3, kernel=(3, 3), pad=(1, 1), name=('%s_3x3' % name))
    cd5x5r = ConvFactoryB(data=data, num_filter=num_d5x5red, kernel=(1, 1), name=('%s_double_3x3' % name), suffix='_reduce')
    cd5x5 = ConvFactoryB(data=cd5x5r, num_filter=num_d5x5, kernel=(5, 5), pad=(2, 2), name=('%s_double_3x3_1' % name))
    pooling = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(1, 1), pad=(1, 1), pool_type=pool, name=('%s_pool_%s_pool' % (pool, name)))
    cproj = ConvFactoryB(data=pooling, num_filter=proj, kernel=(1, 1), name=('%s_proj' %  name))
    concat = mx.symbol.Concat(*[c1x1, c3x3, cd5x5, cproj], name='ch_concat_%s_chconcat' % name)
    return concat

In [24]:
# Inception Batch Normalization Network
#https://arxiv.org/pdf/1502.03167.pdf
data = mx.sym.Variable('data')

conv1 = ConvFactory(data=data, kernel=(3,3), pad=(1,1), num_filter=96, act_type="relu")
in3a = SimpleFactory(conv1, 32, 32)
in3b = SimpleFactory(in3a, 32, 48)
in3c = DownsampleFactory(in3b, 80)
in4a = SimpleFactory(in3c, 112, 48)
in4b = SimpleFactory(in4a, 96, 64)
in4c = SimpleFactory(in4b, 80, 80)
in4d = SimpleFactory(in4c, 48, 96)
in4e = DownsampleFactory(in4d, 96)
in5a = SimpleFactory(in4e, 176, 160)
in5b = SimpleFactory(in5a, 176, 160)
pool = mx.symbol.Pooling(data=in5b, pool_type="avg", kernel=(7,7), name="global_pool")
flatten = mx.symbol.Flatten(data=pool, name="flatten1")
fc = mx.symbol.FullyConnected(data=flatten, num_hidden=62, name="fc1")
ibn = mx.symbol.SVMOutput(data=fc, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart.format = 'png'
network_chart.render('Inception Batch Normalization')

'Inception Batch Normalization.pdf'

<img src = 'Inception Batch Normalization.png'>

In [25]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ibn,         # Use the network we just defined
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, 64),  
    num_epoch = 30,       # Train for 30 epochs
    optimizer_params = {
        'learning_rate': 0.0003,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 907.97 samples/sec	accuracy=0.773077
INFO:root:Epoch[0] Batch [128]	Speed: 977.90 samples/sec	accuracy=0.789795
INFO:root:Epoch[0] Batch [192]	Speed: 980.87 samples/sec	accuracy=0.774658
INFO:root:Epoch[0] Train-accuracy=0.779514
INFO:root:Epoch[0] Time cost=13.614
INFO:root:Epoch[0] Validation-accuracy=0.769991
INFO:root:Epoch[1] Batch [64]	Speed: 947.76 samples/sec	accuracy=0.802885
INFO:root:Epoch[1] Batch [128]	Speed: 997.08 samples/sec	accuracy=0.808350
INFO:root:Epoch[1] Batch [192]	Speed: 920.26 samples/sec	accuracy=0.811768
INFO:root:Epoch[1] Train-accuracy=0.828125
INFO:root:Epoch[1] Time cost=13.487
INFO:root:Epoch[1] Validation-accuracy=0.790901
INFO:root:Epoch[2] Batch [64]	Speed: 950.02 samples/sec	accuracy=0.834375
INFO:root:Epoch[2] Batch [128]	Speed: 995.87 samples/sec	accuracy=0.852539
INFO:root:Epoch[2] Batch [192]	Speed: 972.91 samples/sec	accuracy=0.861572
INFO:root:Epoch[2] Train-accuracy=0.899306
INFO:root:Epoch[2] Time cost=13

INFO:root:Epoch[23] Batch [128]	Speed: 975.31 samples/sec	accuracy=1.000000
INFO:root:Epoch[23] Batch [192]	Speed: 993.00 samples/sec	accuracy=1.000000
INFO:root:Epoch[23] Train-accuracy=1.000000
INFO:root:Epoch[23] Time cost=13.055
INFO:root:Epoch[23] Validation-accuracy=0.938189
INFO:root:Epoch[24] Batch [64]	Speed: 993.52 samples/sec	accuracy=1.000000
INFO:root:Epoch[24] Batch [128]	Speed: 993.50 samples/sec	accuracy=1.000000
INFO:root:Epoch[24] Batch [192]	Speed: 993.70 samples/sec	accuracy=1.000000
INFO:root:Epoch[24] Train-accuracy=1.000000
INFO:root:Epoch[24] Time cost=12.973
INFO:root:Epoch[24] Validation-accuracy=0.938189
INFO:root:Epoch[25] Batch [64]	Speed: 986.10 samples/sec	accuracy=1.000000
INFO:root:Epoch[25] Batch [128]	Speed: 953.55 samples/sec	accuracy=1.000000
INFO:root:Epoch[25] Batch [192]	Speed: 839.73 samples/sec	accuracy=1.000000
INFO:root:Epoch[25] Train-accuracy=1.000000
INFO:root:Epoch[25] Time cost=13.929
INFO:root:Epoch[25] Validation-accuracy=0.938189
INFO

Accuracy: 93.81893382352942 %


In [26]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'Inception_BN'
epoch = 30
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "Inception_BN-0030.params"


In [27]:
modelMetric(mod, 'Finish predict of Inception BN..')

INFO:root:Finish predict of Inception BN..
INFO:root:Test accuracy = 0.938068


             precision    recall  f1-score   support

        0.0       0.93      0.94      0.94      2129
        1.0       0.94      0.93      0.94      2166

avg / total       0.94      0.94      0.94      4295

[[2004  125]
 [ 141 2025]]
